In [38]:
import numpy as np
import pandas as pd
import random as rd
import math
import itertools

def AETC(X, Y, B, c, Q, p, k):
    
    '''
    Adaptive Explore-Then-Commit (AETC) algorithm simulator for multifidelity approximation

    ''' 
    # X is the training dataset (arranged in low/high fidelities)
    # Y is the testing dataset (same order as X)
    # B is the total budget
    # c is the cost vector
    # Q is the risk matrix
    # p is the number of surrogate models
    # k<=p is size limit
    
    # get all models with size less than or equal to k
    m = X.shape[0]
    n = X.shape[1]
    model_list = []
    for j in range(1,k+1):
        model_list = model_list + list(itertools.combinations(range(1,p+1), j))
    
    # exploration cost
    c_epr = sum(c)
    
    # initialization for t
    t = p + 2
    
    # maximal exploration round
    t_max = math.floor(B/sum(c)) - 1
    
    if t>t_max:
        return("Budget is too small")
    elif t_max>m:
        return("Data is deficient")
    else:
        # get training data
        X_tr = np.hstack((np.array([[1]*t_max]).T,X[range(t_max),:]))
        
        for i in range(t,t_max+1):
            # estimated optimal risk vector
            risk_est = [0]*len(model_list)
            
            # exploration vector 
            exp_est = [0]*len(model_list)
            
            # uniform exploration data/design matrix/response matrix
            X_tr_low = X_tr[range(i),:]
            X_dsn = X_tr[range(i),:][:,range(p+1)]
            X_rep = X_tr[range(i),:][:,range(p+1,n+1)]
            
            # estimated covariance matrix/second moments
            Sigma = np.cov(X_dsn.T)
            Lambda = X_dsn.T@X_dsn
            
            # find which model has the optimal expected risk/whether more exploration is need
            for l in range(len(model_list)):
                S = [0] + list(model_list[l])
                Z = X_dsn[:,S]
                c_ept = sum([c[a-1] for a in list(model_list[l])])
                
                # estimators
                Beta_S = np.linalg.pinv(Z)@X_rep
                Beta_S = Beta_S.T
                x_S = np.mean(X_dsn[:,S], axis=0)
                Sigma_S = Sigma[S,:][:,S]
                Lambda_S = np.linalg.inv(Lambda[S,:][:,S])*i
                if n-p==1:
                    Gamma_S = np.array([[np.var((Z@Beta_S.T - X_rep).T)*(i-1)/(i-len(S))]])
                else:
                    Gamma_S = np.cov((Z@Beta_S.T - X_rep).T)*(i-1)/(i-len(S))
                
                # estimation of k1/k2/m_S/risk
                k_1 = c_ept*np.trace(Q@Beta_S@Sigma_S@Beta_S.T@Q.T)
                k_2 = np.trace(Q@Gamma_S@Q.T)*x_S@Lambda_S@x_S.T + 2**(-i)
                m_S = B/(c_epr + math.sqrt(c_epr*k_1/k_2))
                if math.floor(m_S)>i:
                    risk_est[l] = (k_1+c_epr*k_2+2*math.sqrt(k_1*c_epr*k_2))/B
                    exp_est[l] = 1
                else:
                    risk_est[l] = k_1/(B-i*c_epr) + 1/i*k_2
                    exp_est[l] = 0
            
            model = np.argmin(np.array(risk_est))
            if exp_est[model] == 0:
                break
        
        # get the model for exploitation
        S_opt = list(model_list[model])
        Z_opt = X_dsn[:,[0]+S_opt]
        Beta_S = np.linalg.pinv(Z_opt)@X_rep
        res = [S_opt, Beta_S, i, math.floor((B-c_epr*i)/sum([c[a-1] for a in S_opt]))]
        
#         print("Exploitation model:")
#         print(res[0])
#         print("Exploitation samples:")
#         print(res[3])
        
        # total affordable samples
        N = min(res[3], len(Y[:,0]))
        sample_tt = rd.sample(range(len(Y[:,0])), N)
        X_tt = np.hstack((np.array([[1]*res[3]]).T, Y[sample_tt,:][:,[a-1 for a in res[0]]]))
        
        # compute the average of the high fidelity model
        Y_pred = np.mean(X_tt@Beta_S, axis=0)
        return(Y_pred)


In [ ]:
# test of the AETC algorithm on a dataset generated from multilevel parametric PDE solvers

# load the dataset 
mydf = pd.read_csv('/Users/yimingxu/Desktop/AETC/elasticity_data.csv')
mydf = mydf.values.tolist()
mydf = np.array(mydf)
mydf = np.delete(mydf, 0, axis=1)

# 6 low-fidelity models/exhaust all possible models
p = 6
k = 6

# total budget is 1e5
B = 1e5

# cost vector (inverse propto the square of the mesh size, with the lowest model = 1)
c = [4**i for i in range(7)]

# split data into traning (the first 1000 samples) and testing
X = mydf[rd.sample(range(1000),1000),:]
Y = mydf[range(1000,len(mydf[:,0])),:][:,range(p)]

# risk weight matrix set as the identiy matrix
Q = np.diag([1]*(len(mydf[0,:])-p))

res = AETC(X, Y, B, c, Q, p, k)

# AETC
print("AETC estimator:")
res

# MC
print("MC estimator:")
np.mean(mydf[rd.sample(range(len(mydf[:,0])),math.floor(B/c[len(mydf[0,:])-1])),:][:,range(p,len(mydf[0,:]))], axis=0)

print("Oracle:")
# ground truth
np.mean(mydf[:,range(p,len(mydf[0,:]))], axis = 0)
